In [1]:
import numpy as np
import os
import pickle
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import pandas.io.data as web
%matplotlib inline

In [2]:
def get_dataframe():
    mydata = None
    try:
        mydata = pickle.load(open('modelling/stats_pickle5', 'rb'))
    except:
        mydata = pickle.load(open('../resources/stats_pickle5', 'rb'))
    df = pd.DataFrame(mydata, columns=["Time", "Value"])
    df = df.set_index('Time')
    return df

In [3]:
class Name():
    def __init__(self, means, sd):
        self.mean1 = str(means[0])+"hr"
        self.mean1_val = means[0]
        self.mean2 = str(means[1])+"hr"
        self.mean2_val = means[1]
        self.name = self.mean1+"-"+self.mean2
        self.sd = sd
        self.strategy = "Strategy-{}-%.1f".format(self.name) % sd
        self.regime = "Regime-{}-%.1f".format(self.name) % sd
        self.cumsum = "Cumulative-{}-%.1f".format(self.name) % sd
    
    @classmethod
    def from_strategy_name(cls, strat):
        parts = strat.split('-')
        return cls([p.rstrip("hr") for p in parts[1:3]],float(parts[3]))

In [4]:
class StrategySelector():
    def __init__(self, dataframe, name, means, sds):
        self.dataframe = dataframe
        self.current_strategy=None
        self.means = means
        self.create_means()
        self.set_market()
        self.available_stategies = [Name(means, sd) for means in list(itertools.combinations(means, 2)) for sd in SDs]
        for name in regime_and_strategy_names:
            self.create_diff(name)
            self.create_regime(name)
            self.create_strategy(name)
    
    def strategies(self):
        return self.dataframe[[n.strategies for n in self.available_stategies]]
    def create_means(self):
        for mean in self.means:
            self.dataframe[str(mean)+"hr"] = np.round(pd.rolling_mean(df['Value'], window=mean*60), 2)

    def create_diff(self, name):
            self.dataframe[name.name] = self.dataframe[name.mean1] - self.dataframe[name.mean2]

    def create_regime(self, name):
        self.dataframe[name.regime] = np.where(self.dataframe[name.name] > name.sd, 1, 0)
        self.dataframe[name.regime] = np.where(self.dataframe[name.name] < -name.sd, -1, self.dataframe[name.regime])

    def set_market(self):
        self.dataframe['Market'] = np.log(self.dataframe['Value']/ self.dataframe['Value'].shift(1))
        self.dataframe['Market-cumulative'] = self.dataframe['Market'].cumsum().apply(np.exp)

    def create_strategy(self, name):
        self.dataframe[name.strategy] = self.dataframe[name.regime].shift(5) * self.dataframe['Market']
    
    def select_on_time_performance(hours=1):
        dataframe2= self.strategies.cumsum().apply(np.exp) - self.strategies.cumsum().apply(np.exp).shift(hours*60)
        
        #self.dataframe['best_{}_hours'.format(hours)] = self.dataframe[]
        last_24 = df[-2880:][[a.strategy for a in regime_and_strategy_names]].cumsum().apply(np.exp)[-1:].transpose().sort([df.index.tolist()[-1]],ascending=[0])

In [161]:
class Dataset():
    def __init__(self, loc):
        mydata = None
        try:
            mydata = pickle.load(open('modelling/{}'.format(loc), 'rb'))
        except:
            mydata = pickle.load(open('../resources/{}'.format(loc), 'rb'))
        self.dataset = pd.DataFrame(mydata, columns=["Time", "Value"])
        self.dataset = self.dataset.set_index('Time')
        self.dataset['Market'] = np.log(self.dataset['Value']/ self.dataset['Value'].shift(1))
        
    
class Strategy():
    def __init__(self, dataset, means, sd):
        
        self.name = Name(means, sd)
        cols = list(dataset.columns.values)
        if self.name.mean1 not in cols:
            dataset[self.name.mean1] = np.round(pd.rolling_mean(dataset['Value'], window=self.name.mean1_val*60), 2)
        if self.name.mean2 not in cols:
            dataset[self.name.mean2] = np.round(pd.rolling_mean(dataset['Value'], window=self.name.mean2_val*60), 2)
            
        self.dataset = pd.DataFrame(dataset[['Market',self.name.mean1, self.name.mean2]])
        # Find our Diff
        self.dataset['Difference']= self.dataset[self.name.mean1] - self.dataset[self.name.mean2]
        # Set our Regime
        self.dataset['Regime'] = np.where(self.dataset['Difference'] > self.name.sd, 1, 0)
        self.dataset['Regime'] = np.where(self.dataset['Difference'] < -self.name.sd, -1, self.dataset['Regime'])
        self.dataset['Strategy'] = self.dataset['Regime'].shift(1) * self.dataset['Market']
        self.dataset['hourly_gain'] = self.dataset['Strategy'].cumsum().apply(np.exp) - self.dataset['Strategy'].shift(60).cumsum().apply(np.exp)
    
    def cumsum(self, cols):
        return self.dataset[cols].cumsum().apply(np.exp)


    
df = Dataset('stats_pickle5')
df2 = Dataset('stats_pickle5')
st = Strategy(df.dataset, [5,19], 0.3)
st2 = Strategy(df.dataset, [3,18], 0.4)
st3 = Strategy(df.dataset, [4,17], 0.2)
# pan = pd.Panel(items={"5-19":st.dataset,"3-18":st2.dataset,"4-17":st3.dataset}, major_axis=st3.dataset.index, minor_axis=st3.dataset.columns)
a = pd.DataFrame(list("1234"))
b = pd.DataFrame(list("5678"))
c = pd.DataFrame(list("9123"))
pn = pd.Panel({"a":st.dataset,"b":st2.dataset})
f = pn.transpose(2,0,1)[["Regime","hourly_gain"]].to_frame().sort("hourly_gain").transpose().sort()
f
# df.dataset['Strategy'] = 
# df.dataset


# df.dataset['winner'] = np.where(st.dataset["hourly_gain"]> st2.dataset['hourly_gain'], st.dataset['Regime'], st2.dataset['Regime'])
# df.dataset['winner'] = df.dataset['winner']
# df.dataset['Strategy'] = df.dataset['winner'].shift(10) * df.dataset['Market']

# df.dataset[st.name.name]=st.dataset["Strategy"]
# df.dataset[st2.name.name]=st2.dataset["Strategy"]
# df.dataset[['Market',"Strategy",st.name.name,st2.name.name]].cumsum().apply(np.exp).plot(grid=True, figsize=(20,12))

# df.dataset[['Market', 'Strategy',st.name.name,st2.name.name]].plot(grid=True, figsize=(20,12))
# df.dataset[['Market', 'Strategy']].cumsum().apply(np.exp).plot(grid=True, figsize=(20,12))
#df = pd.DataFrame([st,st2], index=np["st","st2"], columns=["Time"])
#df.tail()


major                               a                            \
Time        2015-06-16T17:07:00+00:00 2015-06-16T17:08:00+00:00   
Regime                       1.000000                  1.000000   
hourly_gain                 -0.027384                 -0.027197   

major                               b                            \
Time        2015-06-16T17:07:00+00:00 2015-06-16T17:08:00+00:00   
Regime                       1.000000                  1.000000   
hourly_gain                 -0.027134                 -0.026949   

major                               a                         b  \
Time        2015-06-16T17:09:00+00:00 2015-06-16T17:09:00+00:00   
Regime                       1.000000                  1.000000   
hourly_gain                 -0.026733                 -0.026489   

major                               a                         b  \
Time        2015-06-16T16:56:00+00:00 2015-06-16T16:56:00+00:00   
Regime                       1.000000                  1.000000   
hourly_gain                 -0.025701                 -0.025466   

major                               a                         b  \
Time        2015-06-16T17:06:00+00:00 2015-06-16T17:06:00+00:00   
Regime                       1.000000                  1.000000   
hourly_gain                 -0.024227                 -0.024006   

major                  ...                                       \
Time                   ...            2015-06-16T15:58:00+00:00   
Regime                 ...                             1.000000   
hourly_gain            ...                             0.043468   

major                               a                         b  \
Time        2015-06-16T15:58:00+00:00 2015-06-16T15:57:00+00:00   
Regime                       1.000000                  1.000000   
hourly_gain                  0.043869                  0.046995   

major                                                         a  \
Time        2015-06-16T15:54:00+00:00 2015-06-16T15:57:00+00:00   
Regime                       1.000000                  1.000000   
hourly_gain                  0.047131                  0.047428   

major                               b                         a  \
Time        2015-06-16T15:55:00+00:00 2015-06-16T15:54:00+00:00   
Regime                       1.000000                  1.000000   
hourly_gain                  0.047449                  0.047566   

major                                                         b  \
Time        2015-06-16T15:55:00+00:00 2015-06-16T15:56:00+00:00   
Regime                       1.000000                  1.000000   
hourly_gain                  0.047886                  0.049555   

major                               a  
Time        2015-06-16T15:56:00+00:00  
Regime                       1.000000  
hourly_gain                  0.050012  

[2 rows x 42712 columns]